In [ ]:
# import ee
# ee.Initialize()

# Testing notebook

This notebook is useful to test zonal_stats function. It contains a dictionary with a collection of different formats of data. The function is able to reduce the area of categorical and continuos data from an image or image collection.

In [ ]:
import json
from shapely.geometry import shape

from calculate_area import zonal_stats

In [ ]:
test_data_sources = {
    "categorical": [
        {
            "asset_id":"COPERNICUS/Landcover/100m/Proba-V-C3/Global",
            "ini_date":"2015-01-01",
            "end_date":"2019-12-31",
            "band":"discrete_classification",
            "categorical" : True
        },
    ],
    "continuous": [
        {
            "asset_id":"NASA/MEASURES/GFCC/TC/v3",
            "ini_date":"2015-01-01",
            "end_date":"2019-12-31",
            "band":"tree_canopy_cover",
            "categorical" : False

        },
        {
            "asset_id":"ECMWF/ERA5/MONTHLY",
            "ini_date":"2015-01-01",
            "end_date":"2019-12-31",
            "band":"mean_2m_air_temperature",
            "categorical" : False

        },
        {
            "asset_id":"UCSB-CHG/CHIRPS/PENTAD",
            "ini_date": "2015-01-01",
            "end_date": "2019-12-31",
            "band": "precipitation",
            "categorical" : False
        },
        {
            "asset_id":"CGIAR/SRTM90_V4",
            "ini_date": "2015-01-01",
            "end_date": "2019-12-31",
            "band": "mean_2m_air_temperature",
            "categorical" : False
        },
    ]
}

polygon =     [
        [[12.110134703672, 42.4439359314072],
          [12.2254428638839, 42.2732798542936],
          [12.3407510240958, 42.10262377718],
          [12.4560591843077, 41.9319677000664],
          [12.7005124839569, 41.9319677000664],
          [12.9449657836061, 41.9319677000664],
          [13.1894190832553, 41.9319677000664],
          [13.4338723829045, 41.9319677000664],
          [13.4338723829045, 42.8175343704937],
          [13.2309300209316, 42.8774946138039],
          [13.0279876589586, 42.937454857114],
          [12.8250452969857, 42.9974151004242],
          [12.6221029350128, 43.0573753437344],
          [12.4929577955755, 43.0066397532412],
          [12.3638126561382, 42.9559041627479],
          [12.2346675167009, 42.9051685722547],
          [12.1055223772636, 42.8544329817615],
          [11.9948265434602, 42.6145920085208],
          [12.110134703672, 42.4439359314072]]
    ]

In [ ]:
# Opening JSON file

# Use manual
polygon = """{
    "type": "MultiPolygon",
    "coordinates": [
        [
            [
                [-75.88994979858398, -1.442146588951299],
                [-75.9041976928711, -1.4579343782400327],
                [-75.88651657104492, -1.476982541739627],
                [-75.85647583007812, -1.4534726228737347],
                [-75.88994979858398, -1.442146588951299]
            ]
        ],
        [
            [
                [-75.9100341796875, -1.4939713066293112],
                [-75.88977813720703, -1.5286348420409486],
                [-75.8602523803711, -1.4953441303982713],
                [-75.9100341796875, -1.4939713066293112]
            ]
        ]
    ]
}"""

shape(json.loads(polygon))

In [ ]:
# Or read from file
f = open('sample.geojson')
polygon = json.dumps(json.load(f))
shape(json.loads(polygon))

In [ ]:
test_data = test_data_sources["continuous"][0]

asset_id = test_data["asset_id"]
band = test_data["band"]
ini_date = test_data["ini_date"]
end_date = test_data["end_date"]
categorical = test_data["categorical"]

In [ ]:
result = zonal_stats(asset_id, ini_date, end_date, band, polygon, categorical)

In [ ]:
import ee
ee.Initialize()

In [ ]:
decoded_polygon = json.loads(polygon)

geometry = ee.Geometry.MultiPolygon(decoded_polygon["coordinates"])
source_type = ee.data.getAsset(asset_id)["type"]

image_collection = (
    ee.ImageCollection(asset_id)
      .filter(ee.Filter.date(ini_date, end_date))
      .select(band)
)

In [ ]:
reduced = (image_collection.first()
  .reduceRegion(**{
    "reducer": ee.Reducer.minMax().combine(**{
      "reducer2": ee.Reducer.mean(),
      "sharedInputs": True
    }), 
    "geometry":geometry,
    "scale": 1,
    "bestEffort": True
  }
))

In [ ]:
reduced.getInfo()

In [ ]:
image_collection.first().bandNames().getInfo()

In [ ]:
result

## Test metadata

In [ ]:
import json

In [ ]:
metadata = open('data.json')
json.dumps(json.load(metadata))